# Multi-agent Collaboration for Financial Analysis

learn ways for making agents collaborate with each other.

The libraries are already installed. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL",
    "context": "The task involves using statistical modeling and machine learning to identify trends and predict market movements for the selected stock AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL in order to analyze trends and predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/qu

Final Answer: The current stock price for Apple Inc. (AAPL) is $227.15 with a market cap of $3.434 trillion. The company offers a variety of products such as iPhones, Macs, iPads, wearables, and accessories. AAPL has a profit margin of 23.97% and a return on equity of 157.41%. Additionally, the trailing total returns for AAPL as of 11/21/2024 are as follows: YTD Return 18.56%, 1-Year Return 19.23%, 3-Year Return 43.80%, and 5-Year Return 256.10%.

> Finished chain.
 

The current stock price for Apple Inc. (AAPL) is $227.15 with a market cap of $3.434 trillion. The company offers a variety of products such as iPhones, Macs, iPads, wearables, and accessories. AAPL has a profit margin of 23.97% and a return on equity of 157.41%. Additionally, the trailing total returns for AAPL as of 11/21/2024 are as follows: YTD Return 18.56%, 1-Year Return 19.23%, 3-Year Return 43.80%, and 5-Year Return 256.10%.

Final Answer: 

Insights and alerts about significant market opportunities or threats for

Final Answer: 
After conducting a search on the internet, I found several resources that discuss risk factors and trading strategies for AAPL (Apple Inc.) for users with medium risk tolerance and a preference for Day Trading. Some key points to consider include the importance of risk tolerance in trading decisions, effective risk management strategies for day trading success, and specific day trading strategies for AAPL. Additionally, it is crucial to understand individual preferences, risk tolerance, and market dynamics when developing trading strategies for AAPL. For more detailed information on risk factors and trading strategies for AAPL with medium risk tolerance and a preference for Day Trading, the following resources can be explored: 

1. AAPL (Apple Inc.) Day Trading: Strategies for Success - Vestinda
2. Apple Stock Trading: How to Trade Apple Stocks CFDs - Plus500
3. Effective Risk Management Strategies for Day Trading
4. Apple Stock: 3 Simple Day Trading Strategies | Trading

I need to gather detailed and up-to-date information on approved trading strategies for AAPL, considering current market conditions, optimal pricing, and the user's medium risk tolerance for day trading. I should use the tools available to conduct thorough research and provide a comprehensive analysis.

Action: Read website content
Action Input: {"website_url": "https://www.nasdaq.com/market-activity/stocks/aapl"} 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.nasdaq.com', port=443): Read timed out. (read timeout=15).
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I need to gather information from a different source since I couldn't access the NASDAQ website.
Action: Search the internet
Action Input: {"search_query": "AAPL approved trading strategies"} 


Search results: Title: AAPL Option Strategy Payout Diagram Apple - 

Thought: I need to gather detailed information on AAPL trading strategies from the Saxo website since the Market Chameleon website didn't work.

Action: Read website content
Action Input: {"website_url": "https://www.home.saxo/content/articles/options/mastering-covered-calls---a-strategic-guide-for-apple-investors-03062024"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I should explore another source for detailed information on AAPL trading strategies since the Saxo website content is not accessible.

Action: Search the internet
Action Input: {"search_query": "AAPL trading strategies site:investopedia.com"} 


Search results: Title: Strategies & Applications Behind The 50-Day EMA (INTC, ...
Link: https://www.investopedia.com/articles/active-trading/011415/strategies-applications-behind-50day-ema.asp
Snippet: The 50-day EMA has numerous applications in price prediction, position choice & strategy building.
---
Title:

I need to gather a comprehensive risk analysis report on the proposed trading strategies and execution plans for AAPL to ensure a successful outcome.

Action: Delegate work to co-worker
Action Input: {"coworker": "Risk Advisor", "task": "Evaluate risks associated with covered calls on AAPL", "context": "Detailed Execution Plans for AAPL"}


> Entering new CrewAgentExecutor chain...
I should start by reading the detailed execution plans for AAPL to gather information on the potential risks associated with covered calls on this stock.

Action: Read website content
Action Input: {"website_url": "detailed_execution_plans_AAPL.com"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'detailed_execution_plans_AAPL.com': No scheme supplied. Perhaps you meant https://detailed_execution_plans_AAPL.com?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: Si

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Covered Calls on AAPL:

After evaluating the risks associated with implementing covered calls on AAPL, the following potential risks have been identified:

1. Limited Upside Potential: The maximum gain is limited to the strike price plus the premium received, which caps potential profits if the stock price rises significantly.

2. Downside Risk Exposure: Investors are exposed to the downside risk of the stock if the price decreases below the strike price, potentially resulting in losses.

3. Missed Opportunity for Significant Gains: Engaging in covered calls may result in missing out on substantial gains if the stock price surges significantly above the strike price.

4. Capital Requirements: Implementing covered calls requires a significant amount of capital to purchase the underlying stock, which may limit investment opportunities in other assets.

5. Lower Returns: Covered calls may offer lower returns compared to other investment strategies due to the limited upside potential and the income generated from selling call options.

6. Premium Payments: While selling call options generates premium payments, it also limits the potential for stock appreciation and may result in missed opportunities for higher returns.

To mitigate these risks, investors should consider implementing risk management strategies such as setting appropriate strike prices, selecting optimal expiration dates, and actively monitoring and managing their positions. Conducting thorough research and due diligence before engaging in covered calls is essential to align with risk tolerance and financial goals while maximizing potential returns.